# Panels for figure 2
5/31/24

NOTE: loaded images are aligned to STeg_220429

In [1]:
############ LOAD PACKAGES
import numpy as np
import pandas as pd
import tifffile as tf
import matplotlib.pyplot as plt
import napari

# import custum colormaps
from colormaps import *

# import custum functions
from STP_plotting import *
from STP_processing import *

# auto refreshes imported functions if source code changes:
%load_ext autoreload
%autoreload 2

In [24]:
# set-up vars
# home_dir = "/Volumes/Data/"
home_dir = "/mnt/labNAS/"

in_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/input_tifs/"
out_path = home_dir+"Emily/figures/fig_2_mapseq/napari_stp_screenshots/"

metadata = pd.read_csv("stp_metadata.csv")
meta = metadata[metadata["inj_site"]=="OMC"].reset_index(drop=True)

In [3]:
# import all tiff files aligned to STeg_220429
p05_path = in_path+"norm_subtracted/STeg_220429_aligned/"
p05_images = []
for i in range(meta.shape[0]):
    print("loading:", meta.loc[i, "brain"])
    p05_images.append(tf.imread(p05_path+meta.loc[i,'brain']+"_aligned_to_STeg_220429_NO_subtracted.tif"))
    # p05_images.append(tf.imread(p05_path+meta.loc[i,'brain']+"_p05_norm_subtracted.tif"))


loading: OMC_MMus_220119_b0
loading: OMC_MMus_220303_b1
loading: OMC_MMus_220324_b2
loading: OMC_STeg_220208_b0
loading: OMC_STeg_220411_b1
loading: OMC_STeg_220429_b2


In [4]:
# import masks
# areas made w/ script
areas = ["grey", "CTX", "OMCi", "OMCc", "ACAi", "ACAc", "aud","TH", "STR", "CP", "AMY", "P", "PG", "MB", "PAG", "SCm", 
         "SNr", "HY", "CNU", "TEa", "ECT", "VISC", "AI", "GU", "BS", "HIP"]

# import masks aligned to steg_hand_straightened_asr
masks_path = in_path+"masks/steg_220429_hs_asr/"

# load masks
masks = {}
for area in areas:
 
    with open(masks_path+"steg_220429_hs_asr_"+area+".npy", "rb") as f:
            mask = np.load(f)
    masks[area] = mask


# visualize inverted slices

In [23]:
# visualize w/ napari
viewer = napari.Viewer()

for i in range(len(p05_images)):
    viewer.add_image(
        p05_images[i],
        name=meta.loc[i,"brain"],
        scale=[2.5,1,1],
        colormap="gray_r"
    )

In [6]:
viewer.close()

# inverted individual photos

In [15]:
meta

,brain,species,inj_site,shape
0,OMC_MMus_220119_b0,MMus,OMC,NaN
1,OMC_MMus_220303_b1,MMus,OMC,NaN
2,OMC_MMus_220324_b2,MMus,OMC,NaN
3,OMC_STeg_220208_b0,STeg,OMC,NaN
4,OMC_STeg_220411_b1,STeg,OMC,NaN
5,OMC_STeg_220429_b2,STeg,OMC,NaN


In [28]:
brains = ["OMC_STeg_220411_b1", "OMC_MMus_220119_b0"]
titles = ["OMC", "STR", "AudR_TH_AMY_SNr", "SCm_PAG_PG"]
nslices = [27, 75, 99, 142]
st_contrast = [0, 1457]
mm_contrast = [0, 2174]

for b in brains:
    idx = meta[meta["brain"]==b].index[0]
    im = p05_images[idx]
    sp = meta.loc[idx,"species"]
    if sp=="STeg":
        contrast = st_contrast
    elif sp=="MMus":
        contrast = mm_contrast

    viewer = napari.Viewer()
    viewer.add_image(
        im,
        scale=[2.5,1,1],
        contrast_limits = contrast,
        colormap = "gray_r"
    )

    # add scale bar
    viewer.scale_bar.visible = True
    viewer.scale_bar.unit = "um"

    # take screen shot of each relevant slice
    for i in range(len(nslices)):
        viewer.dims.set_point(0, nslices[i])
        viewer.screenshot(out_path+titles[i]+"_"+b+"_"+str(nslices[i])+".png",
                            canvas_only=True, size=(im.shape[1:]))
    
    viewer.close()


